This introductory course is designed for beginners with little to no prior experience in data science and machine learning (ML). However, a basic understanding of Python syntax and statistics is necessary. The module is divided into three sections: (1) data pre-processing, (2) model training, (3) model evaluation.

The main focus of this module is to teach you how to effectively train an ML model. Therefore, instead of delving deeply into the underlying theories, we will focus on implementation. In the first section, we will cover three popular data pre-processing techniques: standard scaling, principal component analysis, and label encoding. The second section focuses on ML model training and introduces grid search cross-validation, a parameter tuning method to improve model performance. Finally, the third section covers model evaluation, where we assess its effectiveness using metrics like accuracy and the confusion matrix.

# Introduction to Data Science

In [ ]:
pip install seaborn openpyxl

In [ ]:
### Python Libraries ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
### Python Libraries ###

## SECTION 1: Data Pre-processing

Let's begin by reviewing the dataset we'll be working with. A clear understanding of the dataset is essential for making informed decisions regarding model and feature selection. In this tutorial, we will use the Dry Bean Classification dataset, which includes 16 features and 7 distinct classes: Seker, Barbunya, Bombay, Cali, Dermosan, Horoz, and Sira. Features are the input variables that the model uses to make predictions, and classes refer to the categories or labels. For more details about the features and bean types, please visit https://archive.ics.uci.edu/dataset/602/dry+bean+dataset.

#### STEP 1: Load and Read the Dataset

In [ ]:
Dataset = pd.read_excel('Dry_Bean_Dataset.xlsx')        # Load the dataset into a pandas DataFrame [1]
print(Dataset)

#### STEP 2: Seperate Feature and Target

Here, we will separate the dataset into FeatureData (independent variables) and TargetData (dependent variable) segments. In classification, the terms target and class are sometimes used interchangeably, but there is a key difference. Target refers to the actual label in the dataset that the model will learn to predict, while class refers to one of the possible categories the target can belong to.

In [ ]:
TargetData = Dataset['Class']                               # Extract target colum 'Class'
FeatureData = Dataset.drop('Class', axis='columns')         # Only remove the target column [2]

#### STEP 3: Visualize Classes

We create a pie chart to visualize the percentage of each class, helping us better understand the distribution of the target variable in the dataset. This allows us to assess whether there is a significant imbalance among the classes, which could impact model performance in real-world applications. In our case, the classes are not equally distributed, and there is a noticeable gap between Bombay and Dermason. However, the imbalance is not severe enough to cause major concern.

In [ ]:
ClassDir = Counter(TargetData)              # Counter calculates occurrences of each class [3]
y = np.array(list(ClassDir.values()))       # Converting ClassDir to NumPy array for plotting
labels = list(ClassDir.keys())              # Extract the class labels from the counter
plt.pie(y,                                  # Create a pie char [4]
        labels=labels, 
        autopct='%1.1f%%');                 

#### STEP 4: Visualize Features

We create a horizontal bar chart to visualize the mean values of the features in the dataset. This helps us analyze and compare the scale of the features and select appropriate data preprocessing techniques. 

In [ ]:
FeatureMean = FeatureData.mean().to_dict()                          # Create a dictionary with features as keys and their means as values
fig, ax = plt.subplots(figsize=(20,10))                             # Create a horizontal bar chart with a figure size of 20x10
ax.barh(list(FeatureMean.keys()), list(FeatureMean.values()))       # Plot the feature names on the y-axis and their mean values on the x-axis
ax.invert_yaxis()                                                   # Invert the y-axis so that the bars are ordered from top to bottom
for values in ax.patches:
    plt.text(                                                       # Add labels to each bar in the chart
             s=f'{values.get_width():.3g}' if        
             values.get_width()            
             < 1000 else f'{values.get_width():.6g}',
             x=values.get_width(),
             y=values.get_y()+0.5,
             fontsize=10,
             fontweight ='bold')

#### STEP 5: Apply StandardScaler

As you can see in the image above, features such as ConvexArea and Solidity differ greatly in magnitude, with ConvexArea values in the thousands and Solidity values in decimals. This disparity in scale can cause the model to favor ConvexArea simply because its values are larger. To avoid this bias, we apply the StandardScaler to normalize the data. This process ensures that all features are placed on the same scale, allowing the model to treat each feature equally and impartially.

In [ ]:
Scaler = StandardScaler()                                                       # Initialize the StandardScaler and set the output format to pandas DataFrame
SSFeatureData = Scaler.fit_transform(FeatureData)                               # Apply the scaler to the feature data and transform it to standardized values
SSFeatureData = pd.DataFrame(SSFeatureData, columns=FeatureData.columns)        # Converting to pandas dataframe

#### STEP 6: Visualize Features After Standard Scaling

Once you generate the bar chart using the cell below, you will notice a significant shift in the means of all the features. You’ll observe that their magnitudes are now close to zero, with some features that previously had positive values now shifted to negative. To rescale the data, the StandardScaler first subtracts the feature's mean and then divides by its standard deviation. As a result, the mean of some features may become negative.

In [ ]:
FeatureMean = SSFeatureData.mean().to_dict()                        # Create a dictionary with features as keys and their means as  [5]
fig, ax = plt.subplots(figsize=(20,10))                             # Create a horizontal bar chart with a figure size of 20x10
ax.barh(list(FeatureMean.keys()), list(FeatureMean.values()))       # Plot the feature names on the y-axis and their mean values on the x-axis 
ax.invert_yaxis()                                                   # Invert the y-axis so that the bars are ordered from top to bottom 
for values in ax.patches:
    plt.text(s=f'{values.get_width():.3g}',                         # Add labels to each bar in the chart [6]
             x=values.get_width(),
             y=values.get_y()+0.5,
             fontsize=10,
             fontweight ='bold',
             ha='right' if values.get_width() < 0 else 'left')

#### STEP 7: Apply Principal Componenet Analysis (PCA) 

PCA is a data dimensionality reduction technique, which lowers the risk of 
overfitting and speeds up the training process. Overfitting occurs when a model excels 
on training data but performs poorly on test data, often because it has captured noise 
or irrelevant patterns from an abundance of features. By uncovering the essential structure
in the data, PCA reduces the number of features while preserving the model's accuracy.

#### STEP 8: Visualize Feature Correlation before PCA

When you run the code below, it will generate a heatmap illustrating the linear relationships between different features in the dataset. The correlation is measured on a scale from -1 to 1. Values closer to 1 indicate a positive correlation, meaning that as one feature increases, the other feature increases proportionally. Values closer to -1 represent a negative correlation, meaning that as one feature increases, the other decreases proportionally. Values near 0 suggest no linear correlation, indicating no significant relationship between the two features. Ideally, we want feature combinations with correlations close to zero, as this ensures each feature contributes unique information to the model and prevents redundancy.

In [ ]:
plt.figure(figsize=(16, 9))             # Create a figure with a specified size of 16x9 for the heatmap
sns.heatmap(SSFeatureData.corr(),       # Plot a heatmap of the correlation matrix for the standardized feature data [7]
            cmap='coolwarm', 
            annot=True, 
            vmin=-1, 
            vmax=1)
plt.show()                              # Display the heatmap

#### STEP 9: Apply PCA

We set n_components=8 in the PCA function to reduce the dimensionality of the input data to 8 principal components. These components are determined based on the covariance matrix. The value of n_components is selected through trial and error. Each principal component is a linear combination of the original features in the dataset, where the weights (or coefficients) are determined by the amount of variance that component captures in the data.

In [ ]:
PCAnalysis = PCA(n_components=8)                                    # Initialize PCA to reduce the data to 8 principal components
SSAndPCAFeatureData = PCAnalysis.fit_transform(SSFeatureData)       # Fit and transform the standardized feature data using PCA, reducing dimensionality [8]

#### STEP 10: Visualize Feature Correlation After PCA

After generating the image below, you can see that, aside from the diagonal line (which always contains 1s and can be disregarded), all other values are close to 0. This demonstrates that redundant features have been removed from the training dataset, as the principal components are now uncorrelated, capturing unique information.

In [ ]:
plt.figure(figsize=(16, 9))                                 # Create a figure with a specified size of 16x9 for the heatmap
sns.heatmap(pd.DataFrame(SSAndPCAFeatureData).corr(),       # Plot a heatmap of the correlation matrix for the standardized feature data [7]
            cmap='coolwarm', 
            annot=True, 
            vmin=-1, 
            vmax=1)
plt.show()                                                  # Display the heatmap

#### STEP 11: Encode the labels

Label Encoder converts text labels into numbers, which is essential because many machine learning models, including the one used in this tutorial, require numeric input for computation. After running the following code, you’ll see that each text label has been assigned a number. For example, Sira is assigned the number 6, while Cali is assigned the number 2.

In [ ]:
Encoder = LabelEncoder()                                # Initialize a LabelEncoder to convert categorical target labels into numeric values
LETargetData = Encoder.fit_transform(TargetData)        # Fit the encoder to the target data and transform the labels into numeric values [9]
for Idx, LabelName in enumerate(Encoder.classes_):      
    print(f'{LabelName}: {Idx}')

## SECTION 2: Model Training

#### STEP 12: Split the dataset train and test segments

Using the train_test_split function, we divide the data into training and testing sets. Splitting the dataset allows us to test the model on unseen data, which is crucial for evaluating its performance in real-world scenarios. 

By setting test_size=0.2, we allocate 20% of the data for testing, while the remaining 80% is used for training. The random_state=42 ensures the data is shuffled consistently each time, making the results reproducible. Shuffling the data is important to prevent the model from learning unintended patterns based on the order of the data.

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(SSAndPCAFeatureData, LETargetData, test_size=0.2, random_state=42) # [10]

#### STEP 13: Initialize the model

In this tutorial, we will train a Support Vector Classifier (SVC), a classification algorithm from the Support Vector Machine (SVM) family. SVC can handle both linear and non-linear data, making it suitable for various classification tasks. The model works by finding the optimal hyperplane that separates the different classes in the dataset. Its goal is to maximize the margin between classes, which enhances the model's accuracy in making predictions.

In [ ]:
Model = svm.SVC() # [11]

#### STEP 14: Parameter Search

svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, 
shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, 
verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

The svm.SVC() function has many parameters, and finding the optimal combination for these parameters is not straightforward. There’s no mathematical formula to determine the best values, so it often involves trial and error. Manually adjusting each parameter can be very time-consuming. This is where Grid Search becomes useful. As shown in the code below, we first provide a range of values for 'C' and 'kernel' parameters and then use GridSearchCV to automatically test different combinations. This helps streamline the trial-and-error process and identify the best combination for our model.

In [ ]:
# 'Kernel' : Kernel types 'poly', 'rbf', and 'sigmoid' to handle non-linear data.
# 'C': Regularization parameter controlling the balance between training error and generalization.
ParameterGrid = {
    'kernel':['poly', 'rbf', 'sigmoid'], 
    'C':[0.1,3,5,8] 
}

# Model: The machine learning model to tune (e.g., SVC)
# ParameterGrid: The hyperparameter grid defined earlier
# verbose: verbose=4 provides detailed progress updates during the search 
GridSearch = GridSearchCV(Model, ParameterGrid, verbose=4) # [12]

# Fits dataset parameters
GridSearch.fit(xTrain, yTrain)

print("Best Model Parameters: ", GridSearch.best_params_)

The best parameters found are C=8 and kernel='rbf'. GridSearchCV uses 5 cross-validation (CV) folds and calculates the mean score across all 5 folds for each parameter combination. The combination with the highest mean test score is selected as the best. The code below prints the mean score for each combination, and you can see that C=8 and kernel='rbf' achieve the highest score.

In [ ]:
for score, params in zip(GridSearch.cv_results_['mean_test_score'], GridSearch.cv_results_['params']):      # Visualize the mean test scores and their corresponding parameters [13]
    print(f"C={params['C']:<6} | kernel={ params['kernel']:<8} | {score:.10f}")

## SECTION 3: Model Evaluation

#### STEP 15: Calculate Accuracy

We evaluate our model using accuracy using test data, which measures the ratio of correct predictions to the total number of predictions made. This metric helps us understand how well the model is performing by indicating the percentage of correct classifications out of all predictions.

In [ ]:
BestModel = GridSearch.best_estimator_                      # Retrieve the best model from the grid search [12]
Predictions = BestModel.predict(xTest)                      # Use the best model to make predictions on the test data
Predictions = Encoder.inverse_transform(Predictions)        # Convert the predicted numeric labels back to their original categorical form 
yTest = Encoder.inverse_transform(yTest)                    # Convert the true test labels back to their original categorical form [14]
Accuracy = accuracy_score(Predictions, yTest)               # Calculate the accuracy by comparing predictions with the true test labels [15]
print("Model Accuracy: ", Accuracy)

#### STEP 16: Generate Confusion Matrix

A confusion matrix is a performance evaluation tool for classification models that summarizes prediction outcomes. 
It compares the actual class labels (true values) with the predicted labels, providing a detailed breakdown of how 
well the model has performed. The matrix helps identify not only the correct predictions but also any misclassifications,
giving a deeper insight into the model's accuracy and areas for improvement.

In [ ]:
ConfusionMatrix = confusion_matrix(Predictions, yTest)      # Generate the confusion matrix by comparing predictions with the true test labels
Display = ConfusionMatrixDisplay(ConfusionMatrix,           # Create a confusion matrix display with the original class labels [16]
                                 display_labels=labels)          
Fig, Ax = plt.subplots(figsize=(16, 9))                     # Create a figure with a specified size of 16x9 for the confusion matrix plot
Display.plot(ax=Ax);                                        # Plot the confusion matrix on the specified axes

## Hands-on Excersice

Adjust the value of C and visualize the output scores. Observe how performance varies as C increases or decreases, and determine whether there is a value of C that yields better accuracy than C=8.

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(SSAndPCAFeatureData, LETargetData, test_size=0.2, random_state=42)
Model = svm.SVC()

ParameterGrid = {
    'kernel':['poly', 'rbf', 'sigmoid'],        # TODO: Implement the Linear kernel (e.g., 'kernel':['poly', 'rbf', 'sigmoid', 'linear'])
    'C':[0.1,3,5,8]                             # TODO: Experiment with different C values to improve model performance (e.g., 'C':[0.01, 10, 100])
}

GridSearch = GridSearchCV(Model, ParameterGrid, verbose=4)
GridSearch.fit(xTrain, yTrain)

for score, params in zip(GridSearch.cv_results_['mean_test_score'], GridSearch.cv_results_['params']):
    print(f"C={params['C']:<6} | kernel={ params['kernel']:<8} | {score:.10f}")

# Model Prediction
BestModel = GridSearch.best_estimator_                      
Predictions = BestModel.predict(xTest)                      
Predictions = Encoder.inverse_transform(Predictions)        
yTest = Encoder.inverse_transform(yTest)

# Accuracy
Accuracy = accuracy_score(Predictions, yTest)               
print("Model Accuracy: ", Accuracy)

# Confusion Matrix
ConfusionMatrix = confusion_matrix(Predictions, yTest)
Display = ConfusionMatrixDisplay(ConfusionMatrix,display_labels=labels)
Fig, Ax = plt.subplots(figsize=(16, 9))
Display.plot(ax=Ax); 

#### References

[1] https://www.geeksforgeeks.org/reading-excel-file-using-python/ <br>
[2] https://stackoverflow.com/questions/13411544/delete-a-column-from-a-pandas-dataframe <br>
[3] https://www.geeksforgeeks.org/python-counter-objects-elements/ <br>
[4] https://www.w3schools.com/python/matplotlib_pie_charts.asp <br>
[5] https://stackoverflow.com/questions/31037298/pandas-get-column-average-mean <br>
[6] https://www.geeksforgeeks.org/bar-plot-in-matplotlib/ <br>
[7] https://www.geeksforgeeks.org/how-to-create-a-seaborn-correlation-heatmap-in-python/ <br>
[8] https://www.geeksforgeeks.org/principal-component-analysis-with-python/ <br>
[9] https://www.geeksforgeeks.org/ml-label-encoding-of-datasets-in-python/ <br>
[10] https://www.geeksforgeeks.org/how-to-split-the-dataset-with-scikit-learns-train_test_split-function/ <br>
[11] https://scikit-learn.org/dev/modules/generated/sklearn.svm.SVC.html <br>
[12] https://www.geeksforgeeks.org/daskgridsearchcv-a-competitor-for-gridsearchcv/ <br>
[13] https://stackoverflow.com/questions/63584320/print-parameters-and-scores-gridsearchcv <br>
[14] https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.LabelEncoder.html <br>
[15] https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.accuracy_score.html <br>
[16] https://www.w3schools.com/python/python_ml_confusion_matrix.asp <br>
[17] https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.show.html <br>
[18] https://archive.ics.uci.edu/dataset/602/dry+bean+dataset <br>
[19] https://www.semanticscholar.org/paper/Multiclass-classification-of-dry-beans-using-vision-Koklu-Özkan/e84c31138f2f261d15517d6b6bb8922c3fe597a1

#### Authors
1) Nrushad Joshi (ntj@ornl.gov)
2) Suzanne T Parete-Koon (paretekoonst@ornl.gov)
3) Michael A Sandoval
4) Christopher B Fuson